<a href="https://colab.research.google.com/github/crystdang/DABC-Final-G5/blob/Crystina/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-24 02:47:33--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.11’

postgresql-42.2.16. 100%[===================>] 979.38K  6.29MB/s    in 0.2s    

2023-01-24 02:47:33 (6.29 MB/s) - ‘postgresql-42.2.16.jar.11’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://uoft-dabc-final-g5.s3.us-east-2.amazonaws.com/Wellbeing_and_lifestyle_data_Kaggle.csv"
spark.sparkContext.addFile(url)
data_df = spark.read.csv(SparkFiles.get("Wellbeing_and_lifestyle_data_Kaggle.csv"), sep=",", header=True, inferSchema=True)

In [5]:
data_df.show()

+---------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+-----------------------+
|Timestamp|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|       AGE|GENDER|WORK_LIFE_BALANCE_SCORE|
+---------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+-----------------------+
|   7/7/15|        

In [6]:
data_df.columns

['Timestamp',
 'FRUITS_VEGGIES',
 'DAILY_STRESS',
 'PLACES_VISITED',
 'CORE_CIRCLE',
 'SUPPORTING_OTHERS',
 'SOCIAL_NETWORK',
 'ACHIEVEMENT',
 'DONATION',
 'BMI_RANGE',
 'TODO_COMPLETED',
 'FLOW',
 'DAILY_STEPS',
 'LIVE_VISION',
 'SLEEP_HOURS',
 'LOST_VACATION',
 'DAILY_SHOUTING',
 'SUFFICIENT_INCOME',
 'PERSONAL_AWARDS',
 'TIME_FOR_PASSION',
 'WEEKLY_MEDITATION',
 'AGE',
 'GENDER',
 'WORK_LIFE_BALANCE_SCORE']

In [7]:
data_df.printSchema()
# Timestamp and DAILY_STRESS to be converted

root
 |-- Timestamp: string (nullable = true)
 |-- FRUITS_VEGGIES: integer (nullable = true)
 |-- DAILY_STRESS: string (nullable = true)
 |-- PLACES_VISITED: integer (nullable = true)
 |-- CORE_CIRCLE: integer (nullable = true)
 |-- SUPPORTING_OTHERS: integer (nullable = true)
 |-- SOCIAL_NETWORK: integer (nullable = true)
 |-- ACHIEVEMENT: integer (nullable = true)
 |-- DONATION: integer (nullable = true)
 |-- BMI_RANGE: integer (nullable = true)
 |-- TODO_COMPLETED: integer (nullable = true)
 |-- FLOW: integer (nullable = true)
 |-- DAILY_STEPS: integer (nullable = true)
 |-- LIVE_VISION: integer (nullable = true)
 |-- SLEEP_HOURS: integer (nullable = true)
 |-- LOST_VACATION: integer (nullable = true)
 |-- DAILY_SHOUTING: integer (nullable = true)
 |-- SUFFICIENT_INCOME: integer (nullable = true)
 |-- PERSONAL_AWARDS: integer (nullable = true)
 |-- TIME_FOR_PASSION: integer (nullable = true)
 |-- WEEKLY_MEDITATION: integer (nullable = true)
 |-- AGE: string (nullable = true)
 |-- GE

In [8]:
data_df.describe().show()
# no null values

+-------+-----------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+------------+------+-----------------------+
|summary|  Timestamp|    FRUITS_VEGGIES|      DAILY_STRESS|    PLACES_VISITED|       CORE_CIRCLE|SUPPORTING_OTHERS|   SOCIAL_NETWORK|       ACHIEVEMENT|          DONATION|         BMI_RANGE|   TODO_COMPLETED|              FLOW|       DAILY_STEPS|      LIVE_VISION|       SLEEP_HOURS|     LOST_VACATION|    DAILY_SHOUTING|  SUFFICIENT_INCOME|   PERSONAL_AWARDS|  TIME_FOR_PASSION|WEEKLY_MEDITATION|         AGE|GENDER|WORK_LIFE_BALANCE_SCORE|
+-------+-----------+------------------+------------------+------------------+------------------+-----------------+-

In [9]:
data_df.groupBy("Timestamp").count().show()

+-------------+-----+
|    Timestamp|count|
+-------------+-----+
|      9/29/15|   12|
|      1/22/16|    6|
|      10/4/17|   19|
|      7/15/18|    7|
|1/30/19 12:29|    1|
|2/26/19 15:43|    1|
| 3/14/19 6:15|    1|
| 4/20/19 3:51|    1|
| 4/25/19 8:04|    1|
|5/12/19 13:18|    1|
| 5/22/19 2:24|    1|
|5/26/19 13:40|    1|
|7/25/19 17:41|    1|
| 8/31/19 4:08|    1|
|9/20/19 17:42|    1|
| 11/3/19 7:45|    1|
|11/7/19 20:27|    1|
|11/9/19 17:44|    1|
| 1/18/20 7:59|    2|
|2/18/20 18:41|    1|
+-------------+-----+
only showing top 20 rows



In [10]:
from pyspark.sql.functions import regexp_replace
data_df = data_df.withColumn("DATE", regexp_replace("Timestamp", r'(\s\d{1,2}:\d{2})', ''))

In [11]:
data_df.groupBy("DATE").count().show()

+--------+-----+
|    DATE|count|
+--------+-----+
| 9/29/15|   12|
| 1/22/16|    6|
| 10/4/17|   19|
| 7/15/18|    7|
| 2/16/19|    8|
|  6/7/19|   12|
|  2/4/20|    7|
| 6/15/20|   10|
| 8/28/20|   10|
| 2/20/16|    8|
| 4/19/16|    8|
|  8/1/16|    4|
|11/10/16|    6|
|11/12/16|    3|
|  2/6/17|   13|
|11/22/17|    8|
| 7/30/18|    3|
| 5/19/19|   10|
|  7/5/19|    4|
| 7/26/19|    4|
+--------+-----+
only showing top 20 rows



In [12]:
data_df.groupby("DAILY_STRESS").count().show()

+------------+-----+
|DAILY_STRESS|count|
+------------+-----+
|           3| 4398|
|           0|  676|
|           5| 2052|
|      1/1/00|    1|
|           1| 2478|
|           4| 2960|
|           2| 3407|
+------------+-----+



In [13]:
clean_df = data_df.filter(data_df.DAILY_STRESS <= 5)
clean_df.groupBy("DAILY_STRESS").count().show()

+------------+-----+
|DAILY_STRESS|count|
+------------+-----+
|           3| 4398|
|           0|  676|
|           5| 2052|
|           1| 2478|
|           4| 2960|
|           2| 3407|
+------------+-----+



In [14]:
clean_df = clean_df.withColumn("DAILY_STRESS_clean", clean_df["DAILY_STRESS"].cast('int')).drop("DAILY_STRESS").withColumnRenamed("DAILY_STRESS_clean", "DAILY_STRESS")
clean_df.printSchema()

root
 |-- Timestamp: string (nullable = true)
 |-- FRUITS_VEGGIES: integer (nullable = true)
 |-- PLACES_VISITED: integer (nullable = true)
 |-- CORE_CIRCLE: integer (nullable = true)
 |-- SUPPORTING_OTHERS: integer (nullable = true)
 |-- SOCIAL_NETWORK: integer (nullable = true)
 |-- ACHIEVEMENT: integer (nullable = true)
 |-- DONATION: integer (nullable = true)
 |-- BMI_RANGE: integer (nullable = true)
 |-- TODO_COMPLETED: integer (nullable = true)
 |-- FLOW: integer (nullable = true)
 |-- DAILY_STEPS: integer (nullable = true)
 |-- LIVE_VISION: integer (nullable = true)
 |-- SLEEP_HOURS: integer (nullable = true)
 |-- LOST_VACATION: integer (nullable = true)
 |-- DAILY_SHOUTING: integer (nullable = true)
 |-- SUFFICIENT_INCOME: integer (nullable = true)
 |-- PERSONAL_AWARDS: integer (nullable = true)
 |-- TIME_FOR_PASSION: integer (nullable = true)
 |-- WEEKLY_MEDITATION: integer (nullable = true)
 |-- AGE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- WORK_LIF

In [15]:
from pyspark.sql.functions import to_date
df = data_df.select([to_date("DATE", "M/d/yy").alias("DATE"),
                          'FRUITS_VEGGIES',
                          'DAILY_STRESS',
                          'PLACES_VISITED',
                          'CORE_CIRCLE',
                          'SUPPORTING_OTHERS',
                          'SOCIAL_NETWORK',
                          'ACHIEVEMENT',
                          'DONATION',
                          'BMI_RANGE',
                          'TODO_COMPLETED',
                          'FLOW',
                          'DAILY_STEPS',
                          'LIVE_VISION',
                          'SLEEP_HOURS',
                          'LOST_VACATION',
                          'DAILY_SHOUTING',
                          'SUFFICIENT_INCOME',
                          'PERSONAL_AWARDS',
                          'TIME_FOR_PASSION',
                          'WEEKLY_MEDITATION',
                          'AGE',
                          'GENDER',
                          'WORK_LIFE_BALANCE_SCORE'])
df.show()

+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+-----------------------+
|      DATE|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|       AGE|GENDER|WORK_LIFE_BALANCE_SCORE|
+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+-----------------------+
|2015-07-07|    

In [16]:
date_df = clean_df.groupBy("DATE").count()
date_df.show()

+--------+-----+
|    DATE|count|
+--------+-----+
| 9/29/15|   12|
| 1/22/16|    6|
| 10/4/17|   19|
| 7/15/18|    7|
| 2/16/19|    8|
|  6/7/19|   12|
|  2/4/20|    7|
| 6/15/20|   10|
| 8/28/20|   10|
| 2/20/16|    8|
| 4/19/16|    8|
|  8/1/16|    4|
|11/10/16|    6|
|11/12/16|    3|
|  2/6/17|   13|
|11/22/17|    8|
| 7/30/18|    3|
| 5/19/19|   10|
|  7/5/19|    4|
| 7/26/19|    4|
+--------+-----+
only showing top 20 rows



In [17]:
df.printSchema()

root
 |-- DATE: date (nullable = true)
 |-- FRUITS_VEGGIES: integer (nullable = true)
 |-- DAILY_STRESS: string (nullable = true)
 |-- PLACES_VISITED: integer (nullable = true)
 |-- CORE_CIRCLE: integer (nullable = true)
 |-- SUPPORTING_OTHERS: integer (nullable = true)
 |-- SOCIAL_NETWORK: integer (nullable = true)
 |-- ACHIEVEMENT: integer (nullable = true)
 |-- DONATION: integer (nullable = true)
 |-- BMI_RANGE: integer (nullable = true)
 |-- TODO_COMPLETED: integer (nullable = true)
 |-- FLOW: integer (nullable = true)
 |-- DAILY_STEPS: integer (nullable = true)
 |-- LIVE_VISION: integer (nullable = true)
 |-- SLEEP_HOURS: integer (nullable = true)
 |-- LOST_VACATION: integer (nullable = true)
 |-- DAILY_SHOUTING: integer (nullable = true)
 |-- SUFFICIENT_INCOME: integer (nullable = true)
 |-- PERSONAL_AWARDS: integer (nullable = true)
 |-- TIME_FOR_PASSION: integer (nullable = true)
 |-- WEEKLY_MEDITATION: integer (nullable = true)
 |-- AGE: string (nullable = true)
 |-- GENDER: s

In [18]:
# define age groups
age_groups = df.groupBy("AGE").count()
age_groups.show()

+------------+-----+
|         AGE|count|
+------------+-----+
|  51 or more| 3390|
|    36 to 50| 4655|
|    21 to 35| 6108|
|Less than 20| 1819|
+------------+-----+



In [19]:
# arrange from high to low WLB score
high_low = df.orderBy(df["WORK_LIFE_BALANCE_SCORE"].desc())
high_low.show()

+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+
|      DATE|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|         AGE|GENDER|WORK_LIFE_BALANCE_SCORE|
+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+
|2015-10-1

In [20]:
# arrange from low to high WLB score
low_high = df.orderBy(df["WORK_LIFE_BALANCE_SCORE"])
low_high.show()

+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+
|      DATE|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|         AGE|GENDER|WORK_LIFE_BALANCE_SCORE|
+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+
|2016-10-2

In [21]:
# WLB score average, min, max
from pyspark.sql.functions import avg, min, max
df.select(avg("WORK_LIFE_BALANCE_SCORE"), min("WORK_LIFE_BALANCE_SCORE"), max("WORK_LIFE_BALANCE_SCORE")).show()

+----------------------------+----------------------------+----------------------------+
|avg(WORK_LIFE_BALANCE_SCORE)|min(WORK_LIFE_BALANCE_SCORE)|max(WORK_LIFE_BALANCE_SCORE)|
+----------------------------+----------------------------+----------------------------+
|           666.7515026296032|                       480.0|                       820.2|
+----------------------------+----------------------------+----------------------------+



In [22]:
df.groupBy("GENDER").count().show()

+------+-----+
|GENDER|count|
+------+-----+
|Female| 9858|
|  Male| 6114|
+------+-----+



In [23]:
# female participants
female_df = df.filter(df["GENDER"] == "Female")
female_df.show()

+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+
|      DATE|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|         AGE|GENDER|WORK_LIFE_BALANCE_SCORE|
+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+
|2015-07-0

In [24]:
male_df = df.filter(df["GENDER"] == "Male")
male_df.show()

+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+-----------------------+
|      DATE|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|       AGE|GENDER|WORK_LIFE_BALANCE_SCORE|
+----------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+-----------------------+
|2015-07-08|    

In [25]:
from pyspark.sql.functions import col
final_df = df.drop(col("GENDER")).drop(col("DATE"))
final_df.show()

+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+-----------------------+
|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|       AGE|WORK_LIFE_BALANCE_SCORE|
+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+-----------------------+
|             3|           2|             2|          5|              

In [26]:
import pandas as pd
pandas_df = final_df.toPandas()

In [27]:
pandas_df.head(10)

,FRUITS_VEGGIES,DAILY_STRESS,PLACES_VISITED,CORE_CIRCLE,SUPPORTING_OTHERS,SOCIAL_NETWORK,ACHIEVEMENT,DONATION,BMI_RANGE,TODO_COMPLETED,...,LIVE_VISION,SLEEP_HOURS,LOST_VACATION,DAILY_SHOUTING,SUFFICIENT_INCOME,PERSONAL_AWARDS,TIME_FOR_PASSION,WEEKLY_MEDITATION,AGE,WORK_LIFE_BALANCE_SCORE
0,3,2,2,5,0,5,2,0,1,6,...,0,7,5,5,1,4,0,5,36 to 50,609.5
1,2,3,4,3,8,10,5,2,2,5,...,5,8,2,2,2,3,2,6,36 to 50,655.6
2,2,3,3,4,4,10,3,2,2,2,...,5,8,10,2,2,4,8,3,36 to 50,631.6
3,3,3,10,3,10,7,2,5,2,3,...,0,5,7,5,1,5,2,0,51 or more,622.7
4,5,1,3,3,10,4,2,4,2,5,...,0,7,0,0,2,8,1,5,51 or more,663.9
5,3,2,3,9,10,10,2,3,1,6,...,10,8,0,2,2,10,8,3,51 or more,722.3
6,4,2,10,6,10,10,3,5,2,8,...,5,7,10,0,2,10,8,10,51 or more,727.2
7,3,4,5,3,5,7,4,0,1,8,...,10,6,0,2,2,8,2,2,21 to 35,676.2
8,5,3,6,4,3,3,5,4,1,10,...,5,10,0,2,2,10,3,10,21 to 35,702.8
9,4,4,2,6,10,10,0,4,2,3,...,0,6,0,0,1,3,8,1,51 or more,634.4


In [28]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dabc-final-g5.cnjpfxvbwyqz.us-east-1.rds.amazonaws.com:5432/dabc-final-g5"
config = {"user":"postgres", 
          "password": "DataAnalytics", 
          "driver":"org.postgresql.Driver"}


In [29]:
# Write DataFrame to lifestyle_wellbeing table in RDS

# data_df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)